In [ ]:
import pandas as pd  
import numpy as np
import requests 
import re
import io
import datetime
import tabula
from bs4 import BeautifulSoup
from tabula import read_pdf

### Webscraping unnamed table data

#### Url's

http://floridalottery.com/exptkt/ff.htm

In [ ]:
# Fantasy5
url = ('http://floridalottery.com/exptkt/ff.htm') 

http://floridalottery.com/exptkt/l6.htm

In [ ]:
# Lotto
# url = ('http://floridalottery.com/exptkt/l6.htm')

#### Request object

In [ ]:
r = requests.get(url)
print(r.status_code) # 200 Good to Go!

#### Soup object

In [ ]:
soup = BeautifulSoup(r.text.strip(), 'html.parser') 

#### Parse entire html page, all tables!

In [ ]:
# Returns the entire page! 
table_data = []

for node in soup.find_all('td'): 
    table_data.append(''.join(node.find_all(text=True)))

#### Remove empty unwanted spaces from list.

In [ ]:
while("" in table_data):
    table_data.remove("")

#### Remove ("-") values from lists.

In [ ]:
while("-" in table_data):
    table_data.remove("-")

#### List to ndarray

In [ ]:
arr = np.array(table_data)
print(arr.shape)

#### Converting 1D array to 2D array.

In [ ]:
# len of array / num of cols
x = int(len(arr)/6)
arr2 = np.reshape(arr, (int(x), 6))# Fantasy Five
print(arr2)

In [ ]:
arr2.shape

In [ ]:
type(arr2)

#### Convert remaining table_data to pandas DataFrame

#### Fantasy Five

In [ ]:
Fantasy_Five = pd.DataFrame(arr2, columns= ['Date', 'bal1', 
                                            'bal2', 'bal3',
                                            'bal4', 'bal5'])
Fantasy_Five.head()

#### Convert remaining strings to pandas DateTime object<br>
if not datetime object drop row in Date column, and set Date column as index.

In [ ]:
Fantasy_Five['Date'] = pd.to_datetime(Fantasy_Five['Date'], 
                                      format='%m/%d/%y', 
                                      errors= 'coerce')

In [ ]:
Fantasy_Five = Fantasy_Five.set_index('Date')

In [ ]:
Fantasy_Five.head()

In [ ]:
# Remove Date NaT values by the row
Fantasy_Five = Fantasy_Five.loc[Fantasy_Five.index.notnull()]
Fantasy_Five.head()

In [ ]:
# Object Dtypes
Fantasy_Five.info()

#### Cast all columns to int32

In [ ]:
Fantasy_Five = Fantasy_Five.astype({
    'bal1': 'int32',
    'bal2': 'int32',
    'bal3': 'int32',
    'bal4': 'int32',
    'bal5': 'int32',
})

In [ ]:
Fantasy_Five.info()

#### Cast all DataFrames to .csv or .json files

In [ ]:
# last wrote 10/11/21
Fantasy_Five.to_csv('FantasyFive_csv') 
Fantasy_Five.to_json('FantasyFive_json')

In [ ]:
Fantasy5_Prediction = pd.read_csv('FantasyFive_csv', index_col= 'Date')
Fantasy5_Prediction.head()

#### Passed winners and winning locale<br>
#### Tabula requires java <br>

In [ ]:
header = ['Draw Date', 
          'Winner', 
          'Lottery Retailer', 
          'Annual Payment/Cash Option',
         'Prize Amount',
         'Quick Pick',
         'Top Prize Winning Tickets']
FantasyPDF = tabula.read_pdf('f5of36winners_Dec09_to_present.pdf',
                             pages="all", 
                            guess= False,
                            pandas_options={'header': header})
FantasyPDF

#### Winners Data only goes back to the year 2012

In [ ]:
arr3 = np.array(FantasyPDF, dtype= 'object')
print(arr3.shape)

In [ ]:
# x = int(len(arr3)/7)
arr4 = np.reshape(arr3, (92, 7))# Fantasy Five
print(arr4)

In [ ]:
Fantasy5Winners = pd.DataFrame(FantasyPDF, header= header)
Fantasy5Winners

#### PDF to csv

### EDA 

In [ ]:
Fantasy5_Prediction.info()

In [ ]:
Fantasy5_Prediction.describe()

In [ ]:
Fantasy5_Prediction.bal5.value_counts()